In [19]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.10.1


In [20]:
# Get the current working directory
# cwd = os.getcwd()

X, y = [], []
path = "wheat_leaf"
class_dict = {'Healthy':0, 
              'septoria':1, 
              'stripe_rust':2}

for each_class in class_dict.keys():
    # Get all the files in the path directory
    files = os.listdir(path + '/' + each_class)
    # print(files)
    for each_file in files:
        if each_file[0] == '.':
            continue
            
#         image = tf.keras.utils.load_img(path + '/' + each_class + '/' + each_file, 
#                                         target_size=(224, 224))
#         image_arr = tf.keras.utils.img_to_array(image)
        img = cv.imread(path + '/' + each_class + '/' + each_file)
        image_arr = cv.resize(img, (224,224))
        X.append(image_arr)
        y.append(class_dict[each_class])
    print('successfully loaded for class:',  each_class)

KeyboardInterrupt: 

In [ ]:
X, y = np.array(X), np.array(y)
X.shape, y.shape

((407, 224, 224, 3), (407,))

In [ ]:
# y_train.value_counts()
unique_values, counts = np.unique(y, return_counts=True)
unique_values, counts

(array([0, 1, 2]), array([102,  97, 208], dtype=int64))

In [ ]:
(counts / len(y) * 100).round(2)

array([25.06, 23.83, 51.11])

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=96)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((305, 224, 224, 3), (102, 224, 224, 3), (305,), (102,))

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
# y_train.value_counts()
unique_values, counts = np.unique(y_train, return_counts=True)
unique_values, counts

(array([0, 1, 2]), array([ 77,  75, 153], dtype=int64))

In [ ]:
(counts / len(y_train) * 100).round(2)

array([25.25, 24.59, 50.16])

In [ ]:
# y_train.value_counts()
unique_values, counts = np.unique(y_test, return_counts=True)
unique_values, counts

(array([0, 1, 2]), array([25, 22, 55], dtype=int64))

In [ ]:
(counts / len(y_test) * 100).round(2)

array([24.51, 21.57, 53.92])

In [ ]:
# Convert labels to one-hot encoding
num_classes = 3
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
y_train.shape, y_test.shape

((305, 3), (102, 3))

In [ ]:
###############

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation parameters
train_datagen = ImageDataGenerator(
    rotation_range=20,      # Random rotation within the range [-20, 20] degrees
    width_shift_range=0.1,  # Random horizontal shift by up to 10% of the image width
    height_shift_range=0.1, # Random vertical shift by up to 10% of the image height
    zoom_range=0.2,         # Random zoom by up to 20%
    horizontal_flip=True    # Random horizontal flipping
)

In [ ]:
from tensorflow.keras import layers, models

# Define the CNN architecture
def create_cnn(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    # model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [ ]:
# Create the CNN model
input_shape = X_train.shape[1:]
num_classes = 3
model = create_cnn(input_shape, num_classes)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 26, 26, 64)      

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=25, 
                    batch_size=64, validation_data=(X_test, y_test))

# # Fit the model with data augmentation
# batch_size = 64
# epochs = 10

# history = model.fit(train_datagen.flow(X_train, y_train, batch_size=batch_size),
#                     epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/25
5/5 [==============================] - 8s 2s/step - loss: 1.0661 - accuracy: 0.5016 - val_loss: 0.9858 - val_accuracy: 0.5392
Epoch 2/25
5/5 [==============================] - 8s 2s/step - loss: 0.9767 - accuracy: 0.5016 - val_loss: 0.9310 - val_accuracy: 0.7059
Epoch 3/25
5/5 [==============================] - 9s 2s/step - loss: 0.8167 - accuracy: 0.6951 - val_loss: 0.8293 - val_accuracy: 0.6863
Epoch 4/25
5/5 [==============================] - 9s 2s/step - loss: 0.7216 - accuracy: 0.6623 - val_loss: 0.8226 - val_accuracy: 0.5686
Epoch 5/25
5/5 [==============================] - 9s 2s/step - loss: 0.7043 - accuracy: 0.6656 - val_loss: 0.6971 - val_accuracy: 0.6275
Epoch 6/25
5/5 [==============================] - 9s 2s/step - loss: 0.5707 - accuracy: 0.7410 - val_loss: 0.5930 - val_accuracy: 0.7353
Epoch 7/25
5/5 [==============================] - 9s 2s/step - loss: 0.5029 - accuracy: 0.7902 - val_loss: 0.8309 - val_accuracy: 0.6765
Epoch 8/25
5/5 [=========================

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

4/4 [==============================] - 1s 133ms/step - loss: 0.1993 - accuracy: 0.9412
Test loss: 0.1992722898721695
Test accuracy: 0.9411764740943909


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def dense_block(x, num_layers, growth_rate):
    """Creates a dense block with multiple dense layers."""
    for _ in range(num_layers):
        # Batch normalization
        x = layers.BatchNormalization()(x)
        # ReLU activation
        x = layers.Activation('relu')(x)
        # Convolutional layer
        x = layers.Conv2D(growth_rate, kernel_size=(3, 3), padding='same')(x)
        # Concatenate the input with the output of each convolutional layer
        x = layers.Concatenate()([x, x])
    return x

def transition_layer(x, compression_factor):
    """Creates a transition layer to reduce the number of channels."""
    num_channels = int(x.shape[-1])
    # Batch normalization
    x = layers.BatchNormalization()(x)
    # ReLU activation
    x = layers.Activation('relu')(x)
    # Convolutional layer for channel reduction
    x = layers.Conv2D(num_channels * compression_factor, kernel_size=(1, 1), padding='same')(x)
    # Average pooling for spatial reduction
    x = layers.AveragePooling2D(pool_size=(2, 2), strides=2)(x)
    return x

def create_densenet(input_shape, num_classes, num_dense_blocks=3, num_layers_per_block=4, growth_rate=32, compression_factor=0.5):
    """Creates a DenseNet model."""
    # Input layer
    inputs = tf.keras.Input(shape=input_shape)
    
    # Initial convolutional layer
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
    
    # Dense blocks and transition layers
    for i in range(num_dense_blocks):
        x = dense_block(x, num_layers_per_block, growth_rate)
        if i != num_dense_blocks - 1:
            x = transition_layer(x, compression_factor)
    
    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Fully connected layer
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    # Create the model
    temp_model = models.Model(inputs=inputs, outputs=outputs)
    
    return temp_model

# Define input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape for image classification
num_classes = 3  # Example number of classes

# Create the DenseNet model
densenet1_model = create_densenet(input_shape, num_classes)

# Print the model summary
# densenet1_model.summary()

In [ ]:
# Compile the model
densenet1_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

In [ ]:
# Train the model
history = densenet1_model.fit(X_train, y_train, epochs=15, 
                             batch_size=64, validation_data=(X_test, y_test))

Epoch 1/15
6/6 [==============================] - 43s 7s/step - loss: 1.1116 - accuracy: 0.3908 - val_loss: 1.0887 - val_accuracy: 0.4268
Epoch 2/15
6/6 [==============================] - 52s 9s/step - loss: 1.0511 - accuracy: 0.4677 - val_loss: 1.0687 - val_accuracy: 0.4268
Epoch 3/15
6/6 [==============================] - 50s 8s/step - loss: 0.9347 - accuracy: 0.5631 - val_loss: 1.0263 - val_accuracy: 0.4268
Epoch 4/15
6/6 [==============================] - 44s 7s/step - loss: 0.8900 - accuracy: 0.6062 - val_loss: 1.0564 - val_accuracy: 0.4268
Epoch 5/15
6/6 [==============================] - 45s 7s/step - loss: 0.7353 - accuracy: 0.6554 - val_loss: 1.1126 - val_accuracy: 0.4268
Epoch 6/15
6/6 [==============================] - 45s 7s/step - loss: 0.7684 - accuracy: 0.6215 - val_loss: 0.9825 - val_accuracy: 0.4390
Epoch 7/15
6/6 [==============================] - 47s 8s/step - loss: 0.7134 - accuracy: 0.6554 - val_loss: 1.0652 - val_accuracy: 0.4390
Epoch 8/15
6/6 [==================

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def residual_block(x, filters, strides=1, use_projection=False):
    """Creates a residual block with skip connection."""
    shortcut = x
    if use_projection:
        # Use projection shortcut to match dimensions
        shortcut = layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    
    # First convolutional layer
    x = layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # Second convolutional layer
    x = layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    # Add shortcut to the output of the second convolutional layer
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

def create_resnet(input_shape, num_classes, num_blocks=[2], initial_filters=64):
    """Creates a ResNet model."""
    inputs = tf.keras.Input(shape=input_shape)
    
    # Initial convolutional layer
    x = layers.Conv2D(initial_filters, kernel_size=3, strides=1, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=1, padding='same')(x)
    
    # Residual blocks
    filters = initial_filters
    for i, num_layers in enumerate(num_blocks):
        # First block of each stage with strides=2 for downsampling
        x = residual_block(x, filters=filters, strides=1, use_projection=True)
        for _ in range(1, num_layers):
            # Additional blocks in each stage with strides=1
            x = residual_block(x, filters=filters)
        filters *= 2  # Double the number of filters for the next stage
    
    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Fully connected layer
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    # Create the model
    rn_model = models.Model(inputs=inputs, outputs=outputs)
    
    return rn_model

# Define input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape for image classification
num_classes = 3  # Example number of classes

# Create the ResNet model
resnet_model = create_resnet(input_shape, num_classes)

# Print the model summary
# resnet_model.summary()

In [ ]:
# Compile the model
resnet_model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

In [ ]:
resnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
# Train the model
history = resnet_model.fit(X_train, y_train, epochs=15, 
                           batch_size=64, validation_data=(X_test, y_test))

Epoch 1/15
6/6 [==============================] - 185s 28s/step - loss: 1.0053 - accuracy: 0.5138 - val_loss: 1.0840 - val_accuracy: 0.4268
Epoch 2/15
6/6 [==============================] - 165s 27s/step - loss: 0.8112 - accuracy: 0.6308 - val_loss: 1.0728 - val_accuracy: 0.4268
Epoch 3/15
6/6 [==============================] - 166s 27s/step - loss: 0.6400 - accuracy: 0.7015 - val_loss: 1.0606 - val_accuracy: 0.4268
Epoch 4/15
6/6 [==============================] - 173s 29s/step - loss: 0.5680 - accuracy: 0.7815 - val_loss: 1.0825 - val_accuracy: 0.4268
Epoch 5/15
6/6 [==============================] - 166s 27s/step - loss: 0.5725 - accuracy: 0.7662 - val_loss: 1.2043 - val_accuracy: 0.4268
Epoch 6/15
6/6 [==============================] - 164s 27s/step - loss: 0.6345 - accuracy: 0.7385 - val_loss: 1.2312 - val_accuracy: 0.4268
Epoch 7/15
6/6 [==============================] - 162s 26s/step - loss: 0.5344 - accuracy: 0.7662 - val_loss: 1.3218 - val_accuracy: 0.4268
Epoch 8/15
6/6 [====

: 

In [ ]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.densenet import DenseNet121

# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet')

In [ ]:
# Freeze the convolutional base
base_model.trainable = False

In [ ]:
# Add custom dense layers on top of the convolutional base
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Output layer with reduced size (e.g., 10 classes)
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 1000)              8062504   
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 256)               256256    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               3

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=25, 
                    batch_size=64, validation_data=(X_test, y_test))

Epoch 1/25
6/6 [==============================] - 29s 4s/step - loss: 1.0930 - accuracy: 0.4523 - val_loss: 1.0866 - val_accuracy: 0.4268
Epoch 2/25
6/6 [==============================] - 24s 4s/step - loss: 1.0687 - accuracy: 0.5323 - val_loss: 1.0758 - val_accuracy: 0.4268
Epoch 3/25
6/6 [==============================] - 23s 4s/step - loss: 1.0405 - accuracy: 0.5323 - val_loss: 1.0623 - val_accuracy: 0.4268
Epoch 4/25
6/6 [==============================] - 24s 4s/step - loss: 1.0017 - accuracy: 0.5323 - val_loss: 1.0566 - val_accuracy: 0.4268
Epoch 5/25
6/6 [==============================] - 24s 4s/step - loss: 0.9638 - accuracy: 0.5323 - val_loss: 1.0579 - val_accuracy: 0.4268
Epoch 6/25
6/6 [==============================] - 24s 4s/step - loss: 0.9436 - accuracy: 0.5323 - val_loss: 1.0476 - val_accuracy: 0.4268
Epoch 7/25
6/6 [==============================] - 24s 4s/step - loss: 0.9136 - accuracy: 0.5323 - val_loss: 0.9970 - val_accuracy: 0.4268
Epoch 8/25
6/6 [==================

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

3/3 [==============================] - 4s 1s/step - loss: 0.5443 - accuracy: 0.8049
Test loss: 0.5442630648612976
Test accuracy: 0.8048780560493469


In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.10.1


In [2]:
# Get the current working directory
# cwd = os.getcwd()

X, y = [], []
path = "wheat_leaf"
class_dict = {'Healthy':0, 
              'septoria':1, 
              'stripe_rust':2}

for each_class in class_dict.keys():
    # Get all the files in the path directory
    files = os.listdir(path + '/' + each_class)
    # print(files)
    for each_file in files:
        if each_file[0] == '.':
            continue
            
#         image = tf.keras.utils.load_img(path + '/' + each_class + '/' + each_file, 
#                                         target_size=(224, 224))
#         image_arr = tf.keras.utils.img_to_array(image)
        img = cv.imread(path + '/' + each_class + '/' + each_file)
        image_arr = cv.resize(img, (224,224))
        X.append(image_arr)
        y.append(class_dict[each_class])
    print('successfully loaded for class:',  each_class)

successfully loaded for class: Healthy
successfully loaded for class: septoria
successfully loaded for class: stripe_rust


In [3]:
X, y = np.array(X), np.array(y)
X.shape, y.shape

((407, 224, 224, 3), (407,))

In [4]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=96)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((305, 224, 224, 3), (102, 224, 224, 3), (305,), (102,))

In [5]:
X_train = X_train / 255
X_test = X_test / 255

In [6]:
# y_train.value_counts()
unique_values, counts = np.unique(y_train, return_counts=True)
unique_values, counts

(array([0, 1, 2]), array([ 77,  75, 153], dtype=int64))

In [7]:
# y_train.value_counts()
unique_values, counts = np.unique(y_test, return_counts=True)
unique_values, counts

(array([0, 1, 2]), array([25, 22, 55], dtype=int64))

In [8]:
# Convert labels to one-hot encoding
num_classes = 3
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
y_train.shape, y_test.shape

((305, 3), (102, 3))

In [9]:
##############################

In [10]:
# Define the CNN architecture
def create_cnn(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    # model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [15]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.densenet import DenseNet121

base_model = DenseNet121()
base_model.trainable=False

In [16]:
# Create the CNN model
input_shape = X_train.shape[1:]
num_classes = 3
my_model = create_cnn(input_shape, num_classes)

In [18]:
base_model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_3[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                       

In [22]:
# Concatenate the outputs of DenseNet and custom model
densenet_output = base_model.layers[-1].output
custom_output = my_model.layers[-1].output
concatenated_output = layers.Concatenate()([densenet_output, custom_output])

# Define the merged model
merged_model = models.Model(inputs=[base_model.input, my_model.input], outputs=concatenated_output)

# Compile the merged model
# merged_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Add custom dense layers on top of the convolutional base
model = models.Sequential([
    merged_model,
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Output layer with reduced size (e.g., 10 classes)
])

# Compile the merged model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the summary of the merged model
model.summary()

AssertionError: Exception encountered when calling layer "model_5" (type Functional).

Could not compute output KerasTensor(type_spec=TensorSpec(shape=(None, 1003), dtype=tf.float32, name=None), name='concatenate_5/concat:0', description="created by layer 'concatenate_5'")

Call arguments received by layer "model_5" (type Functional):
  • inputs=tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)
  • training=False
  • mask=None